In [2]:
import os
import time
import datetime
import requests
import pandas as pd
import schedule
from dotenv import load_dotenv

# 1. 환경 변수에서 API 키 로딩
load_dotenv('.env')
SERVICE_KEY = os.getenv("OPENWEATHER_API_KEY")

# 2. 설정값
API_URL = "http://apis.data.go.kr/1360000/WthrWrnInfoService/getWthrWrnList"
ALERT_CSV = "weather_alerts_seoul.csv"
REGION_CODE = "11B10101"

# 3. 기존 특보 불러오기
def load_existing_alerts():
    if os.path.exists(ALERT_CSV):
        return pd.read_csv(ALERT_CSV)
    else:
        return pd.DataFrame()

# 4. 특보 수집
def fetch_alerts():
    all_alerts = []

    # 오늘 날짜
    today = datetime.datetime.now()
    
    # 6일 전부터 오늘까지 반복
    for i in range(6, -1, -1):  # 6,5,4,...,0
        target_date = (today - datetime.timedelta(days=i)).strftime("%Y%m%d") + "0000"
        
        params = {
            "serviceKey": SERVICE_KEY,
            "pageNo": "1",
            "numOfRows": "100",
            "dataType": "JSON",
            "areaCode": REGION_CODE,
            "fromTmFc": target_date
        }

        try:
            response = requests.get(API_URL, params=params)
            response.raise_for_status()
            data = response.json()
            items = data.get('response', {}).get('body', {}).get('items', [])
            print(f"📆 {target_date[:8]} - {len(items)}건 수집됨")
            all_alerts.extend(items)
        except Exception as e:
            print(f"❌ {target_date[:8]} 수집 실패: {e}")

    return all_alerts



# 5. 새 특보만 저장
def save_incremental_alerts(alerts):
    if not alerts:
        print("⚠️ 수집된 특보가 없습니다.")
        return

    df_new = pd.DataFrame(alerts)
    useful_cols = ['stnId', 'areaCode', 'areaNm', 'warnVar', 'warnSt', 'warnStress', 'tmFc', 'tmEf', 'warnMsg']
    df_new = df_new[[col for col in useful_cols if col in df_new.columns]]
    df_new['collect_date'] = datetime.datetime.now().strftime("%Y-%m-%d")

    df_old = load_existing_alerts()

    if not df_old.empty and all(col in df_old.columns for col in ['tmFc', 'warnVar', 'warnStress', 'areaNm']):
        merge_keys = ['tmFc', 'warnVar', 'warnStress', 'areaNm']
        df_merge = pd.merge(df_new, df_old, on=merge_keys, how='left', indicator=True)
        df_filtered = df_merge[df_merge['_merge'] == 'left_only'][df_new.columns]
        print(f"🆕 새로운 특보 {len(df_filtered)}건 저장됨")
    else:
        df_filtered = df_new
        print(f"📦 기존 데이터가 없거나 병합 키 없음 → 전체 {len(df_filtered)}건 저장")

    if not df_filtered.empty:
        write_header = not os.path.exists(ALERT_CSV)
        df_filtered.to_csv(ALERT_CSV, mode='a', header=write_header, index=False)
    else:
        print("✔️ 새로운 특보 없음 (모두 중복)")

    print(f"✅ {datetime.datetime.now().strftime('%Y-%m-%d')} 저장 완료")


# 6. 주기적 실행 함수
def job():
    print(f"\n📥 {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 특보 수집 시작")
    alerts = fetch_alerts()
    save_incremental_alerts(alerts)

# 7. 최초 실행 + 1시간마다 반복 실행
job()


📥 2025-07-14 16:36:11 - 특보 수집 시작
❌ 20250708 수집 실패: Expecting value: line 1 column 1 (char 0)
📆 20250709 - 0건 수집됨
📆 20250710 - 0건 수집됨
📆 20250711 - 0건 수집됨
📆 20250712 - 0건 수집됨
📆 20250713 - 0건 수집됨
📆 20250714 - 0건 수집됨
⚠️ 수집된 특보가 없습니다.
